# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df = pd.read_csv('users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## 2. Разбейте данные на выборки

In [3]:
df_train, df_other = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_other, test_size=0.5, random_state=12345)
print('df_train:', len(df_train))
print('df_valid:', len(df_valid))
print('df_test:', len(df_test))

df_train: 1928
df_valid: 643
df_test: 643


#### Всё разбилось корректно, разбивала по схеме 60,20,20

## 3. Исследуйте модели

In [4]:
#задаём features и target
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

#исследуем дерево
tree_result = pd.DataFrame(columns = ['depth', 'accuracy_train', 'accuracy_valid'])
i= 0
for depth in range(1, 21, 1):
    model_tree = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model_tree.fit(features_train, target_train)
    predictions_tree_train = model_tree.predict(features_train)
    predictions_tree_valid = model_tree.predict(features_valid)
    accuracy_tree_train = accuracy_score(target_train, predictions_tree_train)
    accuracy_tree_valid = accuracy_score(target_valid, predictions_tree_valid)
    tree_result.loc[i, 'depth'] = depth
    tree_result.loc[i, 'accuracy_train'] = accuracy_tree_train
    tree_result.loc[i, 'accuracy_valid'] = accuracy_tree_valid
    i +=1
    
tree_result

,depth,accuracy_train,accuracy_valid
0,1,0.75778,0.754277
1,2,0.787863,0.782271
2,3,0.807573,0.785381
3,4,0.810685,0.77916
4,5,0.820021,0.77916
5,6,0.837656,0.783826
6,7,0.855809,0.782271
7,8,0.862552,0.77916
8,9,0.881224,0.782271
9,10,0.889004,0.774495


In [5]:
#исследуем лес
forest_result = pd.DataFrame(columns = ['estim', 'depth', 'accuracy_train', 'accuracy_valid'])
i= 0
for estim in range(5, 51, 5):
    for depth in range(2, 11, 2):
        model_forest = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345)
        model_forest.fit(features_train, target_train)
        predictions_forest_train = model_forest.predict(features_train)
        predictions_forest_valid = model_forest.predict(features_valid)
        accuracy_forest_train = accuracy_score(target_train, predictions_forest_train)
        accuracy_forest_valid = accuracy_score(target_valid, predictions_forest_valid)
        forest_result.loc[i, 'estim'] = estim
        forest_result.loc[i, 'depth'] = depth
        forest_result.loc[i, 'accuracy_train'] = accuracy_forest_train
        forest_result.loc[i, 'accuracy_valid'] = accuracy_forest_valid
        i +=1
    
forest_result   

,estim,depth,accuracy_train,accuracy_valid
0,5,2,0.760373,0.769829
1,5,4,0.817427,0.785381
2,5,6,0.835581,0.794712
3,5,8,0.866183,0.793157
4,5,10,0.880705,0.780715
5,10,2,0.778527,0.777605
6,10,4,0.814315,0.790047
7,10,6,0.835062,0.800933
8,10,8,0.869813,0.796267
9,10,10,0.885892,0.791602


In [6]:
#исследуем логистическую регрессию
model_log = LogisticRegression(random_state=12345)
model_log.fit(features_train, target_train)
predictions_log_train = model_log.predict(features_train)
predictions_log_valid = model_log.predict(features_valid)
accuracy_log_train = accuracy_score(target_train, predictions_log_train)
accuracy_log_valid = accuracy_score(target_valid, predictions_log_valid)
print('Качество логистической регрессии на обучающей выборке:', accuracy_log_train )
print('Качество логистической регрессии на валидационной выборке:', accuracy_log_valid )

Качество логистической регрессии на обучающей выборке: 0.7479253112033195
Качество логистической регрессии на валидационной выборке: 0.7542768273716952


C:\Users\Tehnodent\DS\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [8]:
#выбираем лучшую модель
print('Максимальное значение качества модели "Дерево" на валидационной выборке:', tree_result['accuracy_valid'].max())
print('Максимальное значение качества модели "Лес" на валидационной выборке:', forest_result['accuracy_valid'].max())
print('Значение качества логистической модели на валидационной выборке:', accuracy_log_valid)

Максимальное значение качества модели "Дерево" на валидационной выборке: 0.7853810264385692
Максимальное значение качества модели "Лес" на валидационной выборке: 0.8087091757387247
Значение качества логистической модели на валидационной выборке: 0.7542768273716952


#### Выбираем лес, т.к. его качество выше остальных. Вторым идёт дерево, и потом уже - логистическая регрессия. Однако, дерево более склонно к переобучению, чем регрессия, так как размах между качеством обучающей модели и качеством валидационной у неё больше всего.

## 4. Проверьте модель на тестовой выборке

In [9]:
#находим индекс строки с максимальным значением, чтобы задать такие же гиперпараметры
max_accuracy = forest_result['accuracy_valid'].max()
print(forest_result.query('accuracy_valid == @max_accuracy').index)

#считаем качество модели на тестовой выборке
model = RandomForestClassifier(n_estimators=40, max_depth=8, random_state=12345)
model.fit(features_train, target_train)
predictions= model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print('Качество модели на тестовой выборке:',accuracy)

Int64Index([38], dtype='int64')
Качество модели на тестовой выборке: 0.7962674961119751


#### Качество модели соответствует требованиям: практически в 80% случаев модель предскажет правильный тариф.

## 5. (бонус) Проверьте модели на адекватность

In [35]:
#попробуем сделать 10 случайных моделей, сравним качество

import numpy as np

sanity_check = RandomForestClassifier(n_estimators=40, max_depth=8, random_state=12345)
sanity_check.fit(features_train, target_train)
predictions_check= np.ones(target_test.shape)
accuracy_check = accuracy_score(target_test, predictions_check)
print('Качество модели на тестовой выборке:',accuracy_check)
         

Качество модели на тестовой выборке: 0.3157076205287714


Модель адекватна, так как ее качество существенно лучше качества baseline модели